In [5]:
# === ONLY NEW IMPORTS ===
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import joblib

# === Load and Preprocess CSV ===
df = pd.read_csv("CICIDS2017_Multiclass_Balanced_5k.csv")
df.columns = df.columns.str.strip()
df = df.dropna(axis=1, how='all')
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# === Label Encoding ===
label_encoder = joblib.load("label_encoder.pkl")
df['Label'] = label_encoder.transform(df['Label'].astype(str))

# === Drop non-numeric columns ===
non_numerics = df.select_dtypes(include=['object']).columns
df = df.drop(non_numerics.difference(['Label']), axis=1)

# === Synthetic Data Generation Placeholder ===
def generate_synthetic_data(target_size, feature_columns):
    np.random.seed(42)
    synthetic_data = np.random.normal(loc=0.0, scale=1.0, size=(target_size, len(feature_columns)))
    return pd.DataFrame(synthetic_data, columns=feature_columns)

# === Load synthetic data generated by GAN ===
df_synthetic = generate_synthetic_data(target_size=5000, feature_columns=df.columns[:-1])
df_synthetic['Label'] = df['Label'].max() + 1  # Label for synthetic attack

# === Combine real + synthetic data ===
feature_cols = df.drop("Label", axis=1).columns.tolist()
df_combined = pd.concat([df, df_synthetic], ignore_index=True)

# === Feature/Label separation and standardization ===
X = df_combined[feature_cols].values
y = df_combined["Label"].values
input_len = len(feature_cols)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")
with open("features_list.pkl", "wb") as f:
    joblib.dump(feature_cols, f)

# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# === Torch Dataloaders ===
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=128)

# === Multiclass CNN Architecture ===
class CNNMulticlassIDS(nn.Module):
    def __init__(self, input_len, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear((input_len // 4) * 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.network(x)

# === Training ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(np.unique(y))
model = CNNMulticlassIDS(input_len=input_len, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
epochs = 15

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

# === Evaluation ===
model.eval()
all_preds = []
all_probs = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        logits = model(xb)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

print("\n\U0001F4CA Multiclass IDS Evaluation:")
print(classification_report(y_test, all_preds))
print("Confusion Matrix:")
print(confusion_matrix(y_test, all_preds))

# === Save TorchScript model ===
example_input = torch.rand(1, 1, input_len).to(device)
traced_model = torch.jit.trace(model, example_input)
traced_model.save("ids_cnn_multiclass.pt")
print("✅ Saved: ids_cnn_multiclass.pt")


Epoch 1/15, Loss: 220.7224
Epoch 2/15, Loss: 67.3000
Epoch 3/15, Loss: 53.0356
Epoch 4/15, Loss: 47.1625
Epoch 5/15, Loss: 44.9223
Epoch 6/15, Loss: 41.3076
Epoch 7/15, Loss: 39.1202
Epoch 8/15, Loss: 37.7883
Epoch 9/15, Loss: 37.3028
Epoch 10/15, Loss: 34.9120
Epoch 11/15, Loss: 34.7965
Epoch 12/15, Loss: 34.1921
Epoch 13/15, Loss: 32.7747
Epoch 14/15, Loss: 30.8445
Epoch 15/15, Loss: 31.6988

📊 Multiclass IDS Evaluation:
              precision    recall  f1-score   support

           0       0.99      0.90      0.94      1000
           1       0.99      1.00      0.99      1000
           2       0.99      1.00      0.99      1000
           3       0.97      1.00      0.98      1000
           4       0.99      0.99      0.99      1000
           5       0.99      0.99      0.99      1000
           6       0.99      0.99      0.99      1000
           7       1.00      1.00      1.00         2
           8       0.75      0.86      0.80         7
           9       0.98      1.0

c:\Users\aadip\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aadip\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aadip\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo